<a href="https://colab.research.google.com/github/Nishk23/Federated_Learning/blob/main/isolated_training_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
import os
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [ ]:
# Set hyperparameters
epochs = 10
batch_size = 64
learning_rate = 0.001
l2_lambda = 0.001
dropout_rate = 0.5
num_clients = 3

In [ ]:
# Data augmentation
transform_train = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=0.2),
    transforms.ColorJitter(brightness=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=32, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [ ]:
# Load CIFAR-10 dataset
cifar10_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

# Split dataset among clients
dataset_length = len(cifar10_dataset)
lengths = [dataset_length // num_clients] * num_clients

# Distribute any remaining items due to division
for i in range(dataset_length % num_clients):
    lengths[i] += 1

# Split the dataset among clients using the calculated lengths
client_datasets = random_split(cifar10_dataset, lengths)

# Test dataset for evaluation (same for all clients)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Function to create data loaders
def create_data_loaders(dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, _ = random_split(dataset, [train_size, test_size])
    train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    return train_loader

In [ ]:
# Custom ResNet model with dropout
class ResNetDropout(nn.Module):
    def __init__(self, dropout_rate):
        super(ResNetDropout, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(self.model.fc.in_features, 10)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Function to train the model
def train_model(model, train_loader, epochs, learning_rate, l2_lambda):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch + 1}/{epochs}")
            for inputs, labels in tepoch:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                tepoch.set_postfix(loss=epoch_loss / len(tepoch))
        scheduler.step()

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    return accuracy, precision, recall, f1

In [ ]:
# Main process for training and evaluating models for each client
isolated_client_performance_metrics = {}

for i, client_dataset in enumerate(client_datasets):
    train_loader = create_data_loaders(client_dataset)
    model = ResNetDropout(dropout_rate=dropout_rate).to(device)
    train_model(model, train_loader, epochs=epochs, learning_rate=learning_rate, l2_lambda=l2_lambda)
    accuracy, precision, recall, f1 = evaluate_model(model, test_loader)

    isolated_client_performance_metrics[f'client_{i+1}'] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"Client {i+1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch 10/10: 100%|██████████| 209/209 [00:15<00:00, 13.16batch/s, loss=0.688]


Client 1 - Accuracy: 0.7833, Precision: 0.7799, Recall: 0.7833, F1 Score: 0.7794


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 10/10: 100%|██████████| 209/209 [00:15<00:00, 13.17batch/s, loss=0.69]


Client 2 - Accuracy: 0.7762, Precision: 0.7773, Recall: 0.7762, F1 Score: 0.7719


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 10/10: 100%|██████████| 209/209 [00:15<00:00, 13.24batch/s, loss=0.693]


Client 3 - Accuracy: 0.7786, Precision: 0.7776, Recall: 0.7786, F1 Score: 0.7749


In [ ]:
# Save the isolated performance metrics to a JSON file
save_path = '/content/drive/My Drive/Federated_Learning'
os.makedirs(save_path, exist_ok=True)
json_file_path = os.path.join(save_path, 'isolated_client_performance_metrics.json')
with open(json_file_path, 'w') as f:
    json.dump(isolated_client_performance_metrics, f, indent=4)

print(f"Isolated performance metrics have been saved to '{json_file_path}'")

Isolated performance metrics have been saved to '/content/drive/My Drive/Federated_Learning/isolated_client_performance_metrics.json'
